In [135]:
import numpy as np
from scipy import linalg
from qutip import *

In [241]:
def FQ(derivativerho, rho):
    a = rho.full()
    q = 2 * derivativerho.full()

    if np.allclose( a, a.dot(a) ) == True:
        print("The density matrix is a pure state.")
        return np.trace( (q.dot(q)).dot(a) )

    # Compute the symmetric logarithmic derivative by solving the continuous Lyapunov equation
    ## Ref: https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve_continuous_lyapunov.html
    L = linalg.solve_continuous_lyapunov(a, q)
    L2 = (L.dot(L))
    #print(L2.dot(a))
    Fisher_Q = np.trace(L2.dot(a))
    return Fisher_Q

In [242]:
# Test for different quantum states

## Test for qubit state

# Define |+> state for qubit

plus = Qobj([[1], [1]]) / np.sqrt(2)
rho = plus * plus.dag()

# Define the evolution of the state as U(theta) = exp(-i theta sigma_z / 2)

def U(theta):
    return (sigmaz()*0.5*-1j*theta).expm()

def rho_i(theta):
    return U(theta) * rho * U(theta).dag()

def derivative_of_rho(theta,h):
    # Compute central finite difference
    drho_dx = (rho_i(theta + h) - rho_i(theta - h)) / (2 * h)
    return drho_dx


def FQ_qubit_1(theta):
    return np.round(FQ(derivative_of_rho(theta, 1e-4), rho_i(theta)),8)

print(FQ_qubit_1(2))

The density matrix is a pure state.
(1+0j)


In [261]:
## Test for coherent state

# Define |alpha> state for coherent state

alpha = 4.0
#N = 2 * 1024
N = 200
coherent_ket = coherent(N, alpha)
rho = coherent_ket * coherent_ket.dag()

# Define the evolution of the state as U(theta) = exp(-i theta a^dagger a)

def U(theta):
    return (num(N) * -1j * theta).expm()

def rho_i(theta):
    return U(theta) * rho * U(theta).dag()

def derivative_of_rho(theta,h):
    # Compute central finite difference
    drho_dx = (rho_i(theta + h) - rho_i(theta - h)) / (2 * h)
    #drho_dx = np.gradient(rho_i(theta).full(), h)    
    return drho_dx

np.round(FQ(derivative_of_rho(5, 0.0001), rho_i(5)),8)

The density matrix is a pure state.


(63.99997931-0j)

In [290]:
## Test for a mixed state

# Define a mixture of coherent states

alpha = 1.0
N = 50
coherent_ket1 = coherent(N, alpha)
rho = coherent_ket1 * coherent_ket1.dag()

alpha = 1.2
coherent_ket2 = coherent(N, alpha)
rho = (coherent_ket2 * coherent_ket2.dag()) * 0.3 + rho * 0.7

# Define the evolution of the state as U(theta) = exp(-i theta a^dagger a)

def U(theta):
    return (num(N) * -1j * theta).expm()

def rho_i(theta):
    return U(theta) * rho * U(theta).dag()

def derivative_of_rho(theta,h):
    # Compute central finite difference
    drho_dx = (rho_i(theta + h) - rho_i(theta - h)) / (2 * h)
    #drho_dx = np.gradient(rho_i(theta).full(), h)    
    return drho_dx


### Error in the FQ calculation

np.round(FQ(derivative_of_rho(2, 0.0001), rho_i(2)),8)




(4.49512435+0.00312512j)

In [276]:
## Test for a mixed qubit state

# Define the evolution of the state as U(theta) = exp(-i theta sigma_z / 2)

def U(theta):
    return (sigmaz()*0.5*-1j*theta).expm()

# Define a mixture of qubit states |+> and |->

plus = Qobj([[1], [1]]) / np.sqrt(2)
rho = plus * plus.dag()

minus = Qobj([[1], [-1]]) / np.sqrt(2)
rho = U(np.pi/3) * (minus * minus.dag()) * U(np.pi/3).dag()  * 0.5 + (plus * plus.dag())*0.5


def rho_i(theta):
    return U(theta) * rho * U(theta).dag()

def derivative_of_rho(theta,h):
    # Compute central finite difference
    drho_dx = (rho_i(theta + h) - rho_i(theta - h)) / (2 * h)
    return drho_dx


def FQ_qubit_mix(theta):
    return np.round(FQ(derivative_of_rho(theta, 1e-4), rho_i(theta)),8)

print(FQ_qubit_mix(3))



(0.25+0j)
